In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Functions
import math

In [ ]:
def Dice(d1,d2):
    #print(set(d1).intersection(d2))
    #print(set(d1).union(set(d2)))
    d=len(set(d1).union(set(d2)))
    if d==0:
        d=1
    return len(set(d1).intersection(d2))/d

In [ ]:
#pairbased
def TestScoreProb(q1,doc1,doc2,i):
    s1 = Dice(q1,doc1)
    #print(s1)
    s2 = Dice(q1,doc2)
    #print(s2)
    p1=0.0
    p2=0.0
    if s1 > s2:
        p1=s1*Prob['less_toxic_prob'][i]
        p2=s2*Prob['more_toxic_prob'][i]
    else:
        p1=s1*Prob['less_toxic_prob'][i]
        p2=s2*Prob['more_toxic_prob'][i]
    s= p1-p2
    d=(1-math.exp(-(s)))
    if d==0:
        return (s1,s2,p1,p2,-0.1,0)
    p = 1/d #(1-math.exp(-(s)))
    if s1 > s2:
        return (s1,s2,p1,p2,p,1)
    return (s1,s2,p1,p2,p,0)

In [ ]:
#pairbased
def TestScoreProb2(s1,s2,n,m):
    p1=0.0
    p2=0.0
    if s1 > s2:
        p1=Prob['less_toxic_prob'][n]
        p2=Prob['more_toxic_prob'][m]
    else:
        p1=Prob['less_toxic_prob'][n]
        p2=Prob['more_toxic_prob'][m]
    s= p1-p2
    d=(1-math.exp(-(s)))
    if d==0:
        return (s1,s2,p1,p2,0)
    p = 1/d #(1-math.exp(-(s)))
    if s1 > s2:
        return (s1,s2,p1,p2,p)
    return (s1,s2,p1,p2,p)

In [ ]:
# import dependencies
%matplotlib inline
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import gensim
from gensim.models import LdaModel
from gensim import models, corpora, similarities
import re
from nltk.stem.porter import PorterStemmer
import time
from nltk import FreqDist
from scipy.stats import entropy
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

In [ ]:
def initial_clean(text):
    """
    Function to clean text of websites, email addresess and any punctuation
    We also lower case the text
    """
    text = re.sub("((\S+)?(http(s)?)(\S+))|((\S+)?(www)(\S+))|((\S+)?(\@)(\S+)?)", " ", text)
    text = re.sub("[^a-zA-Z ]", "", text)
    text = text.lower() # lower case the text
    text = nltk.word_tokenize(text)
    return text

stop_words = stopwords.words('english')
def remove_stop_words(text):
    """
    Function that removes all stopwords from text
    """
    return [word for word in text if word not in stop_words]

stemmer = PorterStemmer()
def stem_words(text):
    """
    Function to stem words, so plural and singular are treated the same
    """
    try:
        text = [stemmer.stem(word) for word in text]
        text = [word for word in text if len(word) > 1] # make sure we have no 1 letter words
    except IndexError: # the word "oed" broke this, so needed try except
        pass
    return text

def apply_all(text):
    """
    This function applies all the functions above into one
    """
    return stem_words(remove_stop_words(initial_clean(text)))

In [ ]:
#train data ../input/jigsaw-toxic-severity-rating/validation_data.csv

#cleaning process
dfTrain = pd.read_csv('../input/jigsaw-toxic-severity-rating/validation_data.csv' ,encoding='utf-8')
dfTrain['Rank'] = [0] * len(dfTrain['less_toxic'])
dfTrain['Label'] = [-1] * len(dfTrain['less_toxic'])
dfTrain.head(20)

dfLexx = dfTrain[["worker", "less_toxic","Rank","Label"]]
dfMore = dfTrain[["worker", "more_toxic","Rank","Label"]]

dfLexx['Label']=[0] * len(dfLexx['less_toxic'])
dfMore['Label']=[1] * len(dfMore['more_toxic'])

dfLexx.rename(columns = {'less_toxic':'text'}, inplace = True)

RankedData = dfLexx
dfMore.rename(columns = {'more_toxic':'text'}, inplace = True)
RankedData3 =RankedData.append(dfMore,ignore_index=True)

In [ ]:
#dfTrain['up'] = dfTrain['less_toxic'] +""+ dfTrain['more_toxic']

In [ ]:
uniqItems = pd.DataFrame(RankedData3['text'].unique(),columns=['text'])

In [ ]:
UniqItemIndx = uniqItems.text.index.values


In [ ]:
# Creates a list containing 5 lists, each of 8 items, all set to 0
w, h = len(UniqItemIndx), len(UniqItemIndx)
#Matrix = [[0 for x in range(w)] for y in range(h)] 

In [ ]:
unikPairs =[]
for p in dfTrain.less_toxic.index.values:
    n =uniqItems[uniqItems.text ==dfTrain.less_toxic[p]].index.values[0]
    m=uniqItems[uniqItems.text ==dfTrain.more_toxic[p]].index.values[0]
    unikPairs.append((n,m))
    #print(uniqItems[uniqItems.text ==dfTrain.less_toxic[p]].index.values[0],uniqItems[uniqItems.text ==dfTrain.more_toxic[p]].index.values[0])
    #Matrix[n][m]=Matrix[n][m]+1
    #break

In [ ]:
#unikPairs = list(set(unikPairs))

In [ ]:
#len(unikPairs)

In [ ]:
# clean text and title and create new column "tokenized"
t1 = time.time()
uniqItems['tokenized'] = uniqItems['text'].apply(apply_all)
t2 = time.time()
print("Time to clean and tokenize", len(dfMore), "articles:", (t2-t1)/60, "min")

In [ ]:
import numpy

In [ ]:
from scipy.sparse import csr_matrix
#A = csr_matrix(Matrix)

In [ ]:
Prob={}
done=[]
a=[]

for i in dfTrain.index.values:
    if i in done:
        continue
    done.append(i)
    nl = dfTrain[dfTrain.less_toxic==dfTrain.less_toxic[i]]
    nm = dfTrain[dfTrain.more_toxic==dfTrain.less_toxic[i]]
    #Prob['less_toxic']=dfTrain.less_toxic[i]
    a.append(len(nl)/(len(nl)+len(nm)))
    
    #break
    
Prob['less_toxic_prob']=a
Prob['less_toxic']=done

In [ ]:
done=[]
a=[]
for i in dfTrain.index.values:
    if i in done:
        continue
    done.append(i)
    nl = dfTrain[dfTrain.less_toxic==dfTrain.more_toxic[i]]
    nm = dfTrain[dfTrain.more_toxic==dfTrain.more_toxic[i]]
    #Prob['less_toxic']=dfTrain.less_toxic[i]
    a.append(len(nm)/(len(nl)+len(nm)))
    #break
    
Prob['more_toxic_prob']=a
Prob['more_toxic']=done

In [ ]:
#cleaning process
dfTest = pd.read_csv('../input/jigsaw-toxic-severity-rating/comments_to_score.csv' ,encoding='utf-8')
dfTest['cleaned_'] = ''
dfTest.head(20)

In [ ]:
# clean text and title and create new column "tokenized"
t1 = time.time()
dfTest['tokenized'] = dfTest['text'].apply(apply_all)
t2 = time.time()
print("Time to clean and tokenize", len(dfLexx), "articles:", (t2-t1)/60, "min")

In [ ]:
len(uniqItems['tokenized'].index.values)
uniqItems['text'][14250]

In [ ]:
# query to doc sim dice
import random
Sim=[]
SimL=[]
T=[]
Label=[]
Models=[]
#SelectedPairs=[]
j=0
DrawSample=2770 #minimum statistical sampling
for x in dfTest['tokenized']:
    i=0
    a=[]
    for pair in random.sample(unikPairs, DrawSample): # y in uniqItems['tokenized']:
        n = pair[0]
        m = pair[1]
        #SelectedPairs.append(pair)
        s1 = Dice(x,uniqItems['tokenized'][pair[0]])
        s2 = Dice(x,uniqItems['tokenized'][pair[1]])
        if s1>s2:
            #print(TestScoreProb2(s1,s2,n,m))
            #break
            T.append(TestScoreProb2(s1,s2,n,m))#(len(x),len(uniqItems['tokenized'][n]),s1,len(uniqItems['tokenized'][m]),s2))
            Label.append(0)
        else:
            T.append(TestScoreProb2(s1,s2,n,m))#(len(x),len(uniqItems['tokenized'][m]),s2,len(uniqItems['tokenized'][n]),s1))
            Label.append(1)
    #Sim.append(T)
    #SimL.append(Label)
    j=j+1
    #if j%3000==0:
        #regLess = LinearRegression().fit(T, Label)
        #Models.append(regLess)
        #T=[]
        #Label=[]
    #break
        

In [ ]:
from sklearn.linear_model import LinearRegression
regLess = LinearRegression().fit(T, Label)

In [ ]:
predictions = regLess.predict(T)

In [ ]:
len(predictions[0:370])

In [ ]:
pre=[]
for i in range(0,len(dfTest['tokenized'])):
    s=i*DrawSample
    e= s+DrawSample
    print(i,s,e)
    t =predictions[s:e]
    #print(dfTest['text'][i])
    print("--------------\n")
    #minpos = t.index(min(t))
    #maxpos = t.index(max(t))
    print(min(t),max(t))
    #print(sum([d for d in t if d>0.0])/len([d for d in t if d>0.0]),sum([d for d in t if d<0.5 and d>0.0])/len([d for d in t if d<0.5 and d>0.0]))
    #print(sum([d for d in t])/len([d for d in t if d<0.5 and d>0.0]))
    pre.append(sum([d for d in t if d>0.0])/len([d for d in t if d>0.0]))


In [ ]:
len(T)/len(dfTest['tokenized'])

#validation
for p in dfTrain.less_toxic.index.values:
    ls = uniqItems[uniqItems.text ==dfTrain.less_toxic[p]].tokenized.iloc[0]
    mr = uniqItems[uniqItems.text ==dfTrain.more_toxic[p]].tokenized.iloc[0]
    q=ls
    s1 = Dice(q,ls)
    s2 = Dice(q,mr)
    validate = TestScoreProb2(s1,s2,n,m)
    tl =regLess.predict([validate])
    #print(t)
    q=mr
    s1 = Dice(q,ls)
    s2 = Dice(q,mr)
    validate = TestScoreProb2(s1,s2,n,m)
    tm = regLess.predict([validate])
    print(tl[0]<tm[0])
    #break
    print("--------------\n")
    #minpos = t.index(min(t))
    #maxpos = t.index(max(t))
    #print(minpos,maxpos)
    #print(sum([d for d in t if d>0.5]),sum([d for d in t if d<0.5]))
    #print(dfTest['tokenized'][i],))
    #break

for i in range(0,len(dfTest['tokenized'])):
    s=i*DrawSample
    e= s+DrawSample
    print(s,e)
    t =regLess.predict(T[s:e])
    print(dfTest['text'][i])
    print("--------------\n")
    #minpos = t.index(min(t))
    #maxpos = t.index(max(t))
    #print(minpos,maxpos)
    print(sum([d for d in t if d>0.5])/len([d for d in t if d>0.5]),sum([d for d in t if d<0.5])/len([d for d in t if d<0.5]))
    #print(dfTest['tokenized'][i],))
    #break

In [ ]:
dataSubmission={}
#dataSubmission['comment_id']=labelsTest
#dataSubmission['score'] = y_pred
dataSubmission['comment_id']=dfTest.comment_id
dataSubmission['score'] = pre

#Train done above

#Train

from sklearn.feature_extraction.text import TfidfVectorizer

tfidfTrain = TfidfVectorizer(sublinear_tf=True, min_df=1, norm='l2', encoding='latin-1', ngram_range=(1, 1),max_features=7000) #stop_words='english' #7000 max features max_features=7000

features = tfidfTrain.fit_transform(RankedData3.text).toarray() 
labels = RankedData3.Rank                           # represents the category of each of the 1490 articles
#ids = df.comment_id

selectedFeatures = tfidfTrain.get_feature_names()


#test

#get common features from train n test
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfTest = TfidfVectorizer(sublinear_tf=True, min_df=1, norm='l2', encoding='latin-1', ngram_range=(1, 1),max_features=12000) #max_features=15000 stop_words='english' ,max_features=12000

featuresTest = tfidfTest.fit_transform(dfTest.text).toarray() 
#featuresTest = tfidfTrain.fit_transform(dfTest.text).toarray() 
labelsTest = dfTest.index.values 

print('d')
intersection = set(tfidfTest.get_feature_names()).intersection(tfidfTrain.get_feature_names())


len(intersection)

filtered=[]
for f in features[0:3000,:]:
    i=0
    for s in f:
        if s>0.1:
            print(selectedFeatures[i],s)
            if selectedFeatures[i] not in filtered:
                filtered.append(selectedFeatures[i])
        i=i+1
len(filtered)

In [ ]:
#list(intersection).index('shut')

In [ ]:
#from sklearn.metrics.pairwise import linear_kernel
#cosine_similarities = linear_kernel(featuresTest[0:1,:7000], features[0:60000,0:7000]) #.flatten()
#t=[for x in cosine_similarities[0] (cosine_similarities[0].index(x),x) if x>0] 
#t=[(idx,val) for idx, val in enumerate(cosine_similarities[0]) if val > 0.1]

In [ ]:
#t

In [ ]:
#print(RankedData3.text[56200],' >> ',RankedData3.Rank[56200])

In [ ]:
#dfTest.text[0]

In [ ]:
#filtered=[]
#for f in features[0:3000,:]:
#    i=0
#    for s in f:
#        if s>0.1:
#            print(selectedFeatures[i],s)
#            if selectedFeatures[i] not in filtered:
#                filtered.append(selectedFeatures[i])
#        i=i+1
#len(filtered)

In [ ]:
#len(intersection)

In [ ]:
#min max rescaling

#max = np.max(labels)
#min = np.min(labels)
#rank_scaled = np.array([(x - min) / (max - min) for x in labels])

In [ ]:
#rank_scaled=100*rank_scaled

In [ ]:
#rank_scaled[0]

from sklearn.preprocessing import Normalizer
transformer = Normalizer().fit([labels])  # fit does nothing.
rank_scaled = transformer.transform([labels])

rank_scaled=[labels]
fNames=tfidfTrain.get_feature_names()


sindexTrain = [fNames.index(x) for x in intersection]

sample = features[: , sindexTrain]
sample.shape
samLab = rank_scaled[0]
samLab.shape

del features

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(sample, samLab, test_size=0.05, random_state=21)

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error

scaler = StandardScaler().fit(sample)
rescaled_X_train = scaler.transform(sample)
model = GradientBoostingRegressor(random_state=21, n_estimators=400,learning_rate=0.09) #subsample=0.5
model.fit(rescaled_X_train, samLab) #normalized y

# transform the validation dataset
rescaled_X_test = scaler.transform(x_test)
predictions = model.predict(rescaled_X_test)
print (mean_squared_error(y_test, predictions))

In [ ]:
#import gc

#del scaler,model

#gc.collect()

for x in model.train_score_:
    print(x)

fNames=tfidfTest.get_feature_names()
sindex = [fNames.index(x) for x in intersection]

#predict
#print(tfidfTrain.get_feature_names())
sampleTest = featuresTest[: , sindex]
sampleTest.shape
samLabTest = labelsTest
samLab.shape

#middle toxic tier ranking
sampleTest=[ftmodel.get_sentence_vector(x) for x in dfTest.text]

#booster
rescaled_X_test = scaler.transform(sampleTest)
predictions = model.predict(rescaled_X_test)

indx = dfTest.text.index.values
for i in range(predictions.shape[0]):
    #print(dfTest.text[indx[i]],predictions[i])
    dfTest.at[indx[i],'Rank']=predictions[i]
    #break

RankedData3 = RankedData3.append(dfTest,ignore_index=True)
#samLab = sample.Rank

#clean
import re
a=[]
for txt in dfTest.text:
    tmp=''
    tmp = txt.replace("_", " ")
    tmp = tmp.replace('"', "")
    tmp = tmp.replace('.', "")
    tmp = tmp.replace("'", "")
    tmp = tmp.replace(":", "")
    tmp = tmp.replace("?", "")
    tmp = tmp.replace(",", "")
    tmp = tmp.replace("!", "")
    tmp = tmp.replace("-", " ")
    tmp = tmp.replace("(", "")
    tmp = tmp.replace(")", "")
    tmp = tmp.replace("[", "")
    tmp = tmp.replace("]", "")
    tmp = tmp.replace("’", "")
    tmp = tmp.replace("}", "")
    tmp = tmp.replace("{", "")
    tmp = tmp.replace("/", "")
    tmp = tmp.replace("=", "")
    tmp = tmp.replace(";", "")
    tmp = tmp.replace("~", "")
    tmp = tmp.replace("^", "")
    tmp = tmp.replace("#", "")
    tmp = tmp.replace(".", "")
    tmp = tmp.replace("=", "")
    tmp = tmp.replace("\n", " ")
    tmp = tmp.replace("http", " weblink ")
    pattern = r'[0-9]'
    tmp = re.sub(pattern, '', tmp)
    
    #print(tmp)
    #print(">>>")
    a.append(tmp)
    #if len(a)==1000:
        #break

dfTest.text=a

from sklearn.feature_extraction.text import TfidfVectorizer

tfidfTrain = TfidfVectorizer(sublinear_tf=True, min_df=1, norm='l2', encoding='latin-1', ngram_range=(1, 1),max_features=7000) #stop_words='english' #7000 max features max_features=7000

features = tfidfTrain.fit_transform(RankedData3.text).toarray() 
labels = RankedData3.Rank                           # represents the category of each of the 1490 articles
#ids = df.comment_id

selectedFeatures = tfidfTrain.get_feature_names()


#test

#get common features from train n test
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfTest = TfidfVectorizer(sublinear_tf=True, min_df=1, norm='l2', encoding='latin-1', ngram_range=(1, 1),max_features=12000) #max_features=15000 stop_words='english' ,max_features=12000

featuresTest = tfidfTest.fit_transform(dfTest.text).toarray() 
#featuresTest = tfidfTrain.fit_transform(dfTest.text).toarray() 
labelsTest = dfTest.comment_id 

print('d')
intersection = set(tfidfTest.get_feature_names()).intersection(tfidfTrain.get_feature_names())


labelsTest = dfTest.comment_id 

print(dfTest.comment_id)

rank_scaled=[labels]
fNames=tfidfTrain.get_feature_names()

sindexTrain = [fNames.index(x) for x in intersection]

sample = features[: , sindexTrain]
sample.shape
samLab = rank_scaled[0]
samLab.shape

del features

[](http://)fNames=tfidfTest.get_feature_names()
sindex = [fNames.index(x) for x in intersection]

#predict
#print(tfidfTrain.get_feature_names())
sampleTest = featuresTest[: , sindex]
sampleTest.shape
samLabTest = labelsTest
samLab.shape

sampleTrain=[ftmodel.get_sentence_vector(x) for x in RankedData3.text]
sampleTest=[ftmodel.get_sentence_vector(x) for x in dfTest.text]

scaler = StandardScaler().fit(sampleTrain)
rescaled_X_train = scaler.transform(sampleTrain)
model = GradientBoostingRegressor(random_state=21, n_estimators=150,learning_rate=0.09) #subsample=0.5
model.fit(rescaled_X_train, RankedData3.Rank) #normalized y

# transform the validation dataset
#rescaled_X_test = scaler.transform(x_test)
#predictions = model.predict(rescaled_X_test)
#print (mean_squared_error(y_test, predictions))

rescaled_X_test = scaler.transform(sampleTest)
predictions = model.predict(rescaled_X_test)

In [ ]:
#test = [" This article sucks woo woo wooooooo"]
#tfidfTest = TfidfVectorizer(sublinear_tf=True, min_df=1, norm='l2', encoding='latin-1', ngram_range=(1, 1),max_features=12000) #max_features=15000 stop_words='english' ,max_features=12000

#featuresTest = tfidfTest.fit_transform(test[0]).toarray() 
#featuresTest = tfidfTrain.fit_transform(dfTest.text).toarray() 
#labelsTest = dfTest.comment_id 


dataSubmission={}
#dataSubmission['comment_id']=labelsTest
#dataSubmission['score'] = y_pred
dataSubmission['comment_id']=labelsTest
dataSubmission['score'] = predictions

In [ ]:
ds= pd.DataFrame.from_dict(dataSubmission)

In [ ]:
ds.to_csv("submission.csv", index=False)